In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from IPython.display import display

#Extra setting
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth',None)
warnings.filterwarnings("ignore")

In [79]:
#Loading train and test data
try:
    train_df=pd.read_csv('/home/anuragverma/Desktop/Kaggle/Titanic/Titanic_classification_pred-1/train.csv')
    test_df=pd.read_csv('/home/anuragverma/Desktop/Kaggle/Titanic/Titanic_classification_pred-1/test.csv')
    print("Train df shape: " ,train_df.shape)
    print("Test df shape: ", test_df.shape)

except FileNotFoundError:
    print('File not loaded')

Train df shape:  (891, 12)
Test df shape:  (418, 11)


In [80]:
#Null check function
def null_zero_check(df):
    null_values = df.isnull().sum()
    zero_values = null_values[null_values == 0]
    Total_record_count=df.shape[0]
    dataype = df.dtypes
    summary_df = pd.DataFrame({'Null count': null_values,
                               'Null_count_percent': null_values/Total_record_count*100,
                                'Zero count': zero_values,
                                'Zero_count_percent': zero_values/Total_record_count*100,
                                'Datatype': dataype})
    return summary_df

#Crosscheck Feature dTypes
def check_feature_dtypes(train_df, test_df):
    common_cols=train_df.columns.intersection(test_df.columns)
    mismatched_dtypes = []
    for col in common_cols:
        if train_df[col].dtype != test_df[col].dtype:
            mismatched_dtypes.append(
                (col, {"train": train_df[col].dtype, "test": test_df[col].dtype})
            )

    # Return True if no mismatches, False otherwise
    if not mismatched_dtypes:
        return True, mismatched_dtypes
    else:
        return False, mismatched_dtypes
    
#Function to Combine train and test data.Remove target from the train data before combining

def combine_df(df1,df2,tgt_col):
    trin_df_drop=df1.drop([tgt_col],axis=1)
    combined_df=pd.concat([trin_df_drop,df2],axis=0)
    return combined_df

#Function to check for duplicates 
def check_duplicates(df, columns=None):
    if columns is None:
        columns = df.columns

    duplicates = df[df.duplicated(subset=columns, keep=False)]
    return duplicates

#Final report generation
def perform_data_analysis(train_df, test_df):
    print('Null check in train_df:')
    display(null_zero_check(train_df))
    print('Null check in test_df:')
    display(null_zero_check(test_df))
    print('Null check in combined_df:')
    combined_df=combine_df(train_df,test_df,'Survived')
    print('Combined_df shape: ',combined_df.shape)
    print('Is rows in train_df + test_df = combined_df?',train_df.shape[0]+test_df.shape[0]==combined_df.shape[0])
    display(null_zero_check(combined_df))

    print("Check describe for train_df")
    display(train_df.describe(include='all').T)
    print("Check describe for test_df")
    display(test_df.describe(include='all').T)
    print("Check describe for combined_df")
    display(combined_df.describe(include='all').T)

    print('Check for dataype mismatch between Train and Test.If True then no mismatch:',check_feature_dtypes(train_df, test_df))

    print('Check for duplicates in train_df')
    display(check_duplicates(train_df))

    print('Check for duplicates in test_df')
    display(check_duplicates(test_df))  

    print('Check for duplicates in combined_df')
    display(check_duplicates(combined_df))

In [82]:
perform_data_analysis(train_df,test_df)

Null check in train_df:


,Null count,Null_count_percent,Zero count,Zero_count_percent,Datatype
Age,177,19.865320,NaN,NaN,float64
Cabin,687,77.104377,NaN,NaN,object
Embarked,2,0.224467,NaN,NaN,object
Fare,0,0.000000,0.0,0.0,float64
Name,0,0.000000,0.0,0.0,object
Parch,0,0.000000,0.0,0.0,int64
PassengerId,0,0.000000,0.0,0.0,int64
Pclass,0,0.000000,0.0,0.0,int64
Sex,0,0.000000,0.0,0.0,object
SibSp,0,0.000000,0.0,0.0,int64


Null check in test_df:


,Null count,Null_count_percent,Zero count,Zero_count_percent,Datatype
Age,86,20.574163,NaN,NaN,float64
Cabin,327,78.229665,NaN,NaN,object
Embarked,0,0.000000,0.0,0.0,object
Fare,1,0.239234,NaN,NaN,float64
Name,0,0.000000,0.0,0.0,object
Parch,0,0.000000,0.0,0.0,int64
PassengerId,0,0.000000,0.0,0.0,int64
Pclass,0,0.000000,0.0,0.0,int64
Sex,0,0.000000,0.0,0.0,object
SibSp,0,0.000000,0.0,0.0,int64


Null check in combined_df:
Combined_df shape:  (1309, 11)
Is rows in train_df + test_df = combined_df? True


...

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Check for duplicates in test_df


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Check for duplicates in combined_df


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
